# Extract

In [1]:
import pymssql
import pandas as pd
import json
import multiprocessing as mp
import datetime
import pymysql

In [2]:
server = '192.168.4.117'
database = 'FreedomCashLenders'
username = 'FreedomCashLendersAll'
mssql_password = 'Freedom123$'


In [3]:
iloans_conn = pymssql.connect(server, username, mssql_password, database, port = 1433)

In [17]:
query_loan = '''select LN.LoanId,
                       LC.LoanCount,
                       LN.OriginationDate,
                       GC.BankReportData,
                       LN.Campaign,
                       LN.MonthlyGrossIncome,
                       LN.DateOfBirth,
                       LN.IsFirstDefault
                       
                from view_FCL_Loan LN
                LEFT JOIN view_FCL_CustomerLoanCount LC ON LC.CustomerId = LN.CustomerId
                LEFT JOIN view_FCL_GetCreditDataLoan GCD ON LN.LoanId = GCD.LoanId
                LEFT JOIN view_FCL_GetCreditData GC ON GC.BankTransactionId = GCD.BankTransactionId
                
                
                where LN.OriginationDate >= '2018-01-01' 
                and LN.OriginationDate <= '2019-12-31' 
                and LN.IsFirstDefault IS NOT NULL
                and LN.MerchantId IN (15, 18)
                and GC.ReportStatus = 'COMPLETE' '''

In [18]:
df_loans = pd.read_sql_query(query_loan,con = iloans_conn)

In [19]:
df_loans = df_loans.drop_duplicates('LoanId')

## Extract Data for Evaluation

In [ ]:
query_loan_eval = '''select LN.LoanId,
                       LC.LoanCount,
                       LN.OriginationDate,
                       GC.BankReportData,
                       LN.Campaign,
                       LN.MonthlyGrossIncome,
                       LN.DateOfBirth,
                       LN.IsFirstDefault
                       
                from view_FCL_Loan LN
                LEFT JOIN view_FCL_CustomerLoanCount LC ON LC.CustomerId = LN.CustomerId
                LEFT JOIN view_FCL_GetCreditDataLoan GCD ON LN.LoanId = GCD.LoanId
                LEFT JOIN view_FCL_GetCreditData GC ON GC.BankTransactionId = GCD.BankTransactionId
                
                
                WHERE
                    GC.ReportStatus = 'COMPLETE'
                    AND LN.LoanId IN %s'''

In [ ]:
df_loan_eval = pd.read_sql_query(query_loan_eval,con=iloans_conn)

# Preprocess

## utility functions

In [8]:
def parse_dates(json_date):
    '''
    Converts json formatted date to pandas datetime.
    
    Parameters:
    JSON date (JSON).
    
    Returns:
    Pandas datetime object.
    
    '''
    
    #return datetime.fromtimestamp(int(json_date)/1000.0).strftime('%Y-%m-%d')
    return datetime.datetime.utcfromtimestamp(int(json_date)/1000).date()


def fetch_checking_acct_txns(json_string):
    """
    Parse all checking account transactions in the bank report
    
    Parameters:
    json_string(json): json containing bank report
    
    Returns:
    dataframe: containing transactions 
    
    """
    j = json.loads(json_string)
    df_txn = pd.DataFrame()
    
    acct_numbers = []
    for accts in j['accounts']:
        
        if ('transactions' in accts.keys()) and (len(accts['transactions']) > 0) and (accts['accountNumber'] not in acct_numbers) and (accts['accountType'].strip().lower() == 'checking'):
            
            df_txn_temp = pd.DataFrame(accts['transactions'])
            df_txn_temp['account_number'] = accts['accountNumber']
            df_txn = df_txn.append(df_txn_temp, ignore_index=True)
            
            df_txn['posted_date'] = df_txn['postedDate'].map(lambda json_date: parse_dates(json_date))
            df_txn['category'] = df_txn['contexts'].map(lambda x: x[0]['categoryName'] if len(x) > 0 else np.nan)
            acct_numbers.append(accts['accountNumber'])
    
    if 'pending' in df_txn.columns:
        df_txn = df_txn[df_txn['pending'] == False]
    return df_txn

## primary account

In [9]:
def get_primary_account(bankreport):
    """
    Flag primary checking account (account having max transaction count)
    
    Parameters:
    bankreport (json)
    loanid (str)
    
    Returns:
    Dataframe containing checking accounts and primary account flag = 1
    """
    df_txn = fetch_checking_acct_txns(bankreport)
    if df_txn.empty is False:
        df_txns_count = df_txn['account_number'].value_counts()
        return df_txns_count.idxmax()

In [20]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df_loans['BankReportData'])

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

In [21]:
df_loans['primary_account'] = res_primary_accts

## filter loans having transaction days >= 60 in primary account

In [22]:
def get_transaction_days_count(primary_account,bank_report):
    df_checking_txns = fetch_checking_acct_txns(bank_report)
    if df_checking_txns.empty is False:
        df_primary_account_txns = df_checking_txns[df_checking_txns['account_number']==primary_account]
        df_primary_account_txns= df_primary_account_txns.sort_values(by='posted_date')
        first_txn_date = df_primary_account_txns['posted_date'].iloc[0]
        last_txn_date = df_primary_account_txns['posted_date'].iloc[-1]
        txn_days_count = (last_txn_date - first_txn_date).days
        return txn_days_count >= 60

In [23]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df_loans['primary_account'],df_loans['BankReportData']))

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

...

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of p

In [24]:
df_loans['txn_days_count'] = txn_days_count

## Calculate Age

In [14]:
def calculate_age(current_date, dob):
    age = len(pd.date_range(start=dob,end=current_date,freq='Y'))
    return age

In [25]:
df_loans['Age'] = df_loans.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

## New or Reloan

In [26]:
df_loans['Reloan'] = df_loans['LoanCount'].apply(lambda x:True if x>1 else False)

# Train

## useful links
https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-deployment-aws.html

https://towardsdatascience.com/deploying-models-to-production-with-mlflow-and-amazon-sagemaker-d21f67909198

https://www.h2o.ai/blog/a-deep-dive-into-h2os-automl/

## specify features

In [ ]:
features= ['LoanId','MonthlyGrossIncome','IsFirstDefault','Age','Reloan']

In [ ]:
df_train = df_loans[features]

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='16G')

In [ ]:
df_h20_train =  h2o.H2OFrame(df_train)

In [ ]:
y = "IsFirstDefault" 
x = df_h20_train.columns
x.remove(y)
x.remove('LoanId')

In [ ]:
aml = H2OAutoML(max_runtime_secs=120, seed=1)
aml.train(x=x, y=y, training_frame=df_h20_train)

In [ ]:
lb = aml.leaderboard
lb.head()

# Predict

### primary account

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df_loan_eval['BankReportData'])

In [ ]:
df_loan_eval['primary_account'] = res_primary_accts

### filter loans having transaction days >= 60 in primary account

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df_loan_eval['primary_account'],df_loan_eval['BankReportData']))

In [ ]:
df_loan_eval['txn_days_count'] = txn_days_count

### calculate age

In [ ]:
df_loan_eval['Age'] = df_loan_eval.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

### new or reloan

In [ ]:
df_loan_eval['Reloan'] = df_loan_eval['LoanCount'].apply(lambda x:True if x>1 else False)

### run prediction

In [ ]:
df_loan_eval = df_loan_eval.loc[df_loan_eval['txn_days_count']==True,:]

In [ ]:
df_loan_eval.reset_index(drop=True,inplace=True)

In [ ]:
features_eval= ['IsFirstDefault','LoanId','MonthlyGrossIncome','Age','Reloan']

In [ ]:
df_loan_eval = df_loan_eval[features_eval]

In [ ]:
h2o_eval = h2o.H2OFrame(df_loan_eval)

In [ ]:
pred = aml.predict(h2o_eval)
pred.head()

In [ ]:
#convert to pandas dataframe
df_predictions = h2o.as_list(pred)

In [ ]:
df_predictions['target'] = df_loan_eval['IsFirstDefault']  

In [ ]:
df_predictions = df_predictions.rename(columns={'True':'prob'})

In [ ]:
df_predictions = df_predictions[['target','prob']]

In [ ]:
df_predictions['target'].value_counts(normalize = True)

# Evaluate

## get BV uncertain and BV Approved loans for model evaluation

In [28]:
username_bank_app = 'bankreview'
password_bank_app = 'Freedom!23'
host_bank_app = '192.168.4.115'
port_bank_app = 3306
db_bank_app = 'bankreviewdb'

In [29]:
bank_app_conn = pymysql.connect(host=host_bank_app,
                                port=port_bank_app,
                                db=db_bank_app,
                                user=username_bank_app,
                                password=password_bank_app)

In [30]:
query_evaluation_loans = '''select loan_id, 
                                final_decision,
                                reasons_for_decision,
                                entered_date
                                
                            from loan 
                            where campaign like '%Production%'
                            and STR_TO_DATE(entered_date ,'%m/%d/%Y') >= STR_TO_DATE('01/01/2020','%m/%d/%Y')
                            and STR_TO_DATE(entered_date ,'%m/%d/%Y') < STR_TO_DATE('04/01/2020','%m/%d/%Y')
                            and final_decision in ('Bank Validation Uncertain','Bank Validation Approved') '''

In [31]:
df_eval_loans = pd.read_sql_query(query_evaluation_loans, con = bank_app_conn)

/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-6-14 20:03:09' for function str_to_date")
  result = self._query(query)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-23 15:26:47' for function str_to_date")
  result = self._query(query)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-23 21:15:04' for function str_to_date")
  result = self._query(query)
/home/amogh/freedom/project_uw_automation/.virtualenvs/production/lib/python3.5/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-26 15:50:50' for function str_to_date")
  result = self._query(query)
/home/amogh/freedom/project_uw_automation/.v

## get funded and mature loans for the same period

In [32]:
query_funded_mature_loans = ''' select LoanId, 
                                IsFirstDefault
                        from view_FCL_Loan
                        where OriginationDate >= '2020-01-01' 
                        and OriginationDate <= '2020-03-31'
                        and IsFirstDefault IS NOT NULL
                        and MerchantId IN (15, 18)
                        
                     '''

In [33]:
df_funded_mature_loans = pd.read_sql_query(query_funded_mature_loans,con = iloans_conn)

In [34]:
df_funded_mature_loans['LoanId'] = df_funded_mature_loans['LoanId'].astype(int).astype(str)

In [35]:
df_eval   = pd.merge(df_funded_mature_loans,df_eval_loans,how = 'inner',left_on = 'LoanId',right_on = 'loan_id')

In [36]:
df_eval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199 entries, 0 to 1198
Data columns (total 6 columns):
LoanId                  1199 non-null object
IsFirstDefault          1199 non-null bool
loan_id                 1199 non-null object
final_decision          1199 non-null object
reasons_for_decision    1199 non-null object
entered_date            1199 non-null object
dtypes: bool(1), object(5)
memory usage: 57.4+ KB


In [ ]:
loan_id_list = list(df_eval['LoanId'])

### compute KS

In [ ]:
import numpy as np

In [ ]:
df_scores = df_predictions.sort_values(by='prob')

In [ ]:
total_good = (df_scores['target'] == False).sum()
total_bad = (df_scores['target'] == True).sum()

In [ ]:
df_scores['cum_good_perc'] = (df_scores['target'] == False).cumsum()/total_good
df_scores['cum_bad_perc'] = (df_scores['target'] == True).cumsum()/total_bad

In [ ]:
df_scores['cum_diff'] = np.abs((df_scores['cum_good_perc'] - df_scores['cum_bad_perc']))
df_scores['cum_diff'].max()